### Process stats for multiband image
Temporary supporting notebook i.e., not intended for use on it's own. Instead it is called from the other notebooks. Only whilst some of the processing chain undergoes changes after which it will be replaced by a python script eventually.

In [ ]:
from parameters.config_imports import *

following makes use of the parameters/lookup_gee_datasets_df.csv

In [ ]:
# get lists for checks
exclude_list = get_exclude_list(lookup_gee_datasets_df)
all_datasets_list = get_all_datasets_list(lookup_gee_datasets_df)
presence_only_flag_list = get_presence_only_flag_list(lookup_gee_datasets_df)
decimal_place_column_list = get_decimal_place_column_list(lookup_gee_datasets_df)
order_list = get_order_list(lookup_gee_datasets_df)

#to make into a checks function 
from modules.datasets import combine_datasets

multiband_image_list = combine_datasets().bandNames()

in_both_lists = multiband_image_list.filter(ee.Filter.inList('item', all_datasets_list));
not_in_multiband = multiband_image_list.filter(ee.Filter.inList('item', all_datasets_list).Not());
not_in_lookup  = ee.List(all_datasets_list).filter(ee.Filter.inList('item', multiband_image_list).Not());

# print ("number_in_multiband_datasets_list", multiband_image_list.length().getInfo())
# print ("number_in_both_lists",in_both_lists.length().getInfo())
# print ("exclude_list",exclude_list)
# print ("not_in_multiband",not_in_multiband.getInfo())

in_lookup = multiband_image_list.containsAll(ee.List(all_datasets_list)).getInfo()

print ("Datasets present in lookup: ", in_lookup)
if in_lookup ==False:
    print("missing from lookup",not_in_lookup.getInfo())

# multiband_datasets_list
# decimal_place_column_list

In [ ]:
# if not present in prefix_columns_list already, remove geo_id_column from it 
# (prefix_columns_list is imported from parameters.config_runtime)
if geo_id_column in prefix_columns_list:
    if geo_id_column in roi.first().propertyNames().getInfo():
        print("geo id present")
    else:
        print("geo id not present, filling with 'na'")
        # Instead of removing, add a column with 'na' as default values
        roi = roi.map(lambda feature: feature.set(geo_id_column, "na"))


column_order_list = prefix_columns_list + order_list

# len (column_order_list)


In [ ]:
st = time.time()

%store -r roi

## if over a certain number of polygons, then batch export to drive and use the formatting for that else, do on the fly

## if batch export to drive then misses out column order # i.e. doesnt do order currently as not possible in a fc, and wont remove underscores as getinfo is client side

## 
if ee.FeatureCollection(roi).size().getInfo()>threshold_to_drive:#update this code to depend on a variable? or if memory error?
    print ("Over size limit for on the fly processing. \n Attempting export to Google Drive 'Whisp_results/Whisp_output_table_export.csv' as a batch task... ")
    
    # stats_fc_formatted = get_stats(roi)
    
    stats_fc_formatted = get_stats_formatted(
    roi,
    # order = column_order_list, # not working as reverts to alphabetical
    id_name = plot_id_column,
    flag_positive = presence_only_flag_list, #,
    # round_properties = decimal_place_column_list#,
    exclude_properties = exclude_list#,
    # select_and_rename = True # underscore removal (DONT RUN - needs debugging relies on get info currently, so only for small runs)
    )

    task = ee.batch.Export.table.toDrive(collection=stats_fc_formatted, 
                     description="whisp_output_table_export", 
                     folder="Whisp_results",
                     fileFormat="CSV")


    task.start()
    
    df_out =  ("Task running. Check https://code.earthengine.google.com/tasks. You can manually upload to Sepal when complete")
    
    print (df_out)
else:
    print ('...stats processing started')
    
    def add_system_index_as_property_to_feature(feature):
        # Get the system:index of the feature
        system_index = feature.get('system:index')
           
        # Set the 'id' property of the feature
        return feature.set('system_index',system_index)
    
    def add_system_index_as_property_to_fc(feature_col):    
       return feature_col.map(add_system_index_as_property_to_feature)

################

    stats_fc_formatted = get_stats_formatted(
    roi,
    # order = column_order_list, # not working as reverts to alphabetical
    id_name = plot_id_column,
    flag_positive = presence_only_flag_list, #,
    # round_properties = decimal_place_column_list#,
    exclude_properties = exclude_list#,
    # select_and_rename = True # underscore removal (DONT RUN - needs debugging relies on get info currently, so only for small runs)
    )
    
    # stats_fc_formatted = get_stats(roi) # if not running formatting

    def add_new_elements(existing_list, new_elements):
        existing_list.extend([element for element in new_elements if element not in existing_list])

    
    if keep_system_index:
        stats_fc_formatted = add_system_index_as_property_to_fc(stats_fc_formatted)
        add_new_elements(column_order_list,['system:index'])

    if keep_original_properties:
        original_columns = roi.first().propertyNames().getInfo()
        add_new_elements(column_order_list,original_columns)

    #convert to dataframe
    df = geemap.ee_to_df(stats_fc_formatted).rename(columns={'system_index':'system:index'})

    # reorder columns 
    df = df.reindex(columns=column_order_list) 
    
    df_out = df  

    print ('...processing time:', time.time() - st, 'seconds')# get the execution time
    